In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.metrics import r2_score
import sys
from sklearn.tree import DecisionTreeRegressor 

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.model_selection import GridSearchCV
import sys
sys.path.append("../Function")


In [67]:
df = pd.read_excel("../../site_info_ver_3.2.xlsx")
df.reset_index(drop=True,inplace=True)
# df = df.dropna(axis = 0)
# dummies = pd.get_dummies(df, 'Site_Name')
# dummies.drop(['Model_number','Date'] , axis = 1, inplace = True)
# dummies = dummies.astype({'Total':np.float64,'Israelis_Count':np.float64, 'Tourists_Count':np.float64})
# dummies.isna().any().sum()

In [68]:
df.columns[25:]
cols_to_drop = ['pm10', 'pm2.5', 'nox', 'so2',
        'Tel_Aviv-Yafo_pm10', 'Jerusalem_pm10',
       'Haifa_pm10', 'Ashkelon_pm10', 'Beer-Sheva_pm10', 'Tel_Aviv-Yafo_pm2.5',
       'Jerusalem_pm2.5', 'Haifa_pm2.5', 'Ashkelon_pm2.5', 'Beer-Sheva_pm2.5',
       'Tel_Aviv-Yafo_nox', 'Jerusalem_nox', 'Haifa_nox', 'Ashkelon_nox',
       'Beer-Sheva_nox', 'Tel_Aviv-Yafo_so2', 'Jerusalem_so2', 'Haifa_so2',
       'Ashkelon_so2', 'Beer-Sheva_so2', 'is_Site_exceeded_pm10',
       'is_Site_exceeded_pm2.5', 'is_Site_exceeded_nox',
       'is_Site_exceeded_so2', 'Tel_Aviv-Yafo_pm10_exceeded',
       'Jerusalem_pm10_exceeded', 'Haifa_pm10_exceeded',
       'Ashkelon_pm10_exceeded', 'Beer-Sheva_pm10_exceeded',
       'Tel_Aviv-Yafo_pm2.5_exceeded', 'Jerusalem_pm2.5_exceeded',
       'Haifa_pm2.5_exceeded', 'Ashkelon_pm2.5_exceeded',
       'Beer-Sheva_pm2.5_exceeded', 'Tel_Aviv-Yafo_so2_exceeded',
       'Jerusalem_so2_exceeded', 'Haifa_so2_exceeded', 'Ashkelon_so2_exceeded',
       'Beer-Sheva_so2_exceeded', 'Tel_Aviv-Yafo_nox_exceeded',
       'Jerusalem_nox_exceeded', 'Haifa_nox_exceeded', 'Ashkelon_nox_exceeded',
       'Beer-Sheva_nox_exceeded']
no_pollution_df = df.drop(columns=cols_to_drop , axis=1)
no_pollution_df.isna().any().sum()

0

In [ ]:
def remove_high_corr(df2,target,threshold):

  target_col = df2.pop(target)
  df2.insert(len(df2.columns), target, target_col)
  cor_matrix = df2.corr().abs()
  corr_df2 = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
  #מתודה שאומרת בי בקורלורציה עם מי
  cols = corr_df2.columns.to_list()
  list_corr_not_empty=[]

  for i in range(len(cols)-1):
      tmp = []
      for j in range(len(cols)-1):
        if abs(corr_df2.iloc[i,j]) >= threshold and cols[i] is not cols[j] :
          tmp.append(cols[j])
      if len(tmp)>0:
          tmp.append(cols[i])
          list_corr_not_empty.append(tmp)
  def key(p):
   return  corr_df2[target][p]
  stay = [max(sub,key=key) for sub in list_corr_not_empty]
  drops = [ c for sub in list_corr_not_empty for c in sub if c not in stay ]
  return list(set(drops))

In [ ]:
def remove_outliers(df,target_name):
  
  plt.cla()
  bp = plt.boxplot(df[target_name])
  minimums = [round(item.get_ydata()[0], 4) for item in bp['caps']][::2]
  maximums = [round(item.get_ydata()[0], 4) for item in bp['caps']][1::2]
  return df.drop(df [ (df[target_name]>maximums[0])  | (df[target_name]<minimums[0])].index)

In [ ]:
def get_outliers(df, series):
  q1 = series.quantile(0.25)
  q3 = series.quantile(0.75)

  if q1*q3 == 0:
    iqr = abs(2*(q1+q3))
    toprange = iqr
    botrange = -toprange
  else:
    iqr = q3-q1
    toprange = q3 + iqr * 1.5
    botrange = q1 - iqr * 1.5

  outliers_top=df[series > toprange]
  outliers_bot= df[series < botrange]
  outliers = pd.concat([outliers_bot, outliers_top], axis=0)

  return (botrange, toprange, outliers)


In [ ]:
total_no_corr = df.drop(remove_high_corr(df,'Total',0.4),axis=1)
israelis_no_corr = df.drop(remove_high_corr(df ,'Israelis_Count',0.4),axis=1)
tourists_no_corr = df.drop(remove_high_corr(df,'Tourists_Count',0.4),axis=1)

to_drop = ['Date','Site_Name']
total_no_corr.drop(to_drop , axis=1 , inplace=True)
israelis_no_corr.drop(to_drop , axis=1 , inplace=True)
tourists_no_corr.drop(to_drop , axis=1 , inplace=True)

botrange, toprange, outliers_total = get_outliers(df, df.Total)
index_total = [i for i in df.index if i not in list(outliers_total.index)]
total_no_corr_no_outliers = total_no_corr.loc[index_total]

botrange, toprange, outliers_israelis = get_outliers(df, df.Israelis_Count)
index_israelis = [i for i in df.index if i not in list(outliers_israelis.index)]
israelis_no_corr_no_outliers = israelis_no_corr.loc[index_israelis]

botrange, toprange, outliers_tourists = get_outliers(df, df.Tourists_Count)
index_tourists = [i for i in df.index if i not in list(outliers_tourists.index)]
tourists_no_corr_no_outliers = tourists_no_corr.loc[index_tourists]

# total_no_corr_no_outliers = remove_outliers(total_no_corr,'Total')
# israelis_no_corr_no_outliers = remove_outliers(israelis_no_corr,'Israelis_Count')
# tourists_no_corr_no_outliers = remove_outliers(tourists_no_corr,'Tourists_Count')

total_no_corr_no_outliers_week = total_no_corr_no_outliers.loc[total_no_corr_no_outliers.is_weekend == 0]
israelis_no_corr_no_outliers_week = israelis_no_corr_no_outliers.loc[israelis_no_corr_no_outliers.is_weekend == 0]
tourists_no_corr_no_outliers_week = tourists_no_corr_no_outliers.loc[tourists_no_corr_no_outliers.is_weekend == 0]

In [ ]:
sites = df.Site_Name.unique()

df.drop(['Date','Site_Name'] , axis=1, inplace=True)

no pollution

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [no_pollution_df.Total , no_pollution_df.Israelis_Count , no_pollution_df.Tourists_Count]


index = 0
for target in targets:

    X = no_pollution_df.drop(['Total' , 'Israelis_Count', 'Tourists_Count' ,'Date', 'Site_Name',] , axis=1)
    
    y = ys[index]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
    train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
    test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
    x_train_scaler = MinMaxScaler()
    x_test_scaler = MinMaxScaler()

    X_train_scaled = x_train_scaler.fit_transform(X_train)
    X_test_scaled = x_test_scaler.fit_transform(X_test)

    y_train_scaled = np.log(y_train + 0.01)
    y_test_scaled = np.log(y_test + 0.01)
    dtree = DecisionTreeRegressor(random_state = 987456) 

    dtree.fit(X_train_scaled,y_train_scaled)
    
    #predict the training data
    y_train_pred_scaled = dtree.predict(X_train_scaled)
    #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
    y_train_pred = np.exp(y_train_pred_scaled)
    #create a pandas series of the results
    y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
    
    #Add the results to the DF
    train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
    #print(train_df.columns)
    y_test_pred_scaled = dtree.predict(X_test_scaled)
    y_test_pred = np.exp(y_test_pred_scaled)
    #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
    #create a pandas series of the results
    y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
    #Add the results to the DF
    test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
    if train_df is None and test_df is None : continue
    train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
    train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
    train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
    train_std = np.std(train_df.predicted_entries_train)
    r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
    #print("Degrees: "+str(i)+",Target: "+target+"\n")
    print("------ TRAIN DATA ------\n")
    print("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
    print("R2 TRAIN "+ str(r2_train)+"\n")
    print("TRAIN STD "+str(train_std)+"\n")
    try:
        test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
    except: pass
    try:
        test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
    except: pass
    try:
        test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
    except: pass
    try:
        r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
    except: pass
    
    test_std = np.std(test_df.predicted_entries_test)
    print("------ TEST DATA ------\n")
    print("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
    print("R2 TEST "+ str(r2_test)+"\n")
    print("TEST STD "+str(test_std)+"\n")
    print("--------------------------------\n")
    
    
    coeff = dtree.feature_importances_
    print(coeff)
    # features = list(X.columns)
    # feat_zip = zip(features,coeff)
    # feat_dict = dict(feat_zip)
    # print(feat_dict)
    #print("EQUATION : " + equation)
    #    new_row = {'Descripton':'general model ,Uni var feature selection, k=10, Degree: '+str(i),'Target':target,'Model_type':'POLY',
    #    'Degree' : str(i),'MAE_Training':train_mae,'MSE_Training':train_mse,
    # 'MSE_Training': train_rmse,'R2_Training': r2_train,
    # 'AE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    # 'EST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}

    #poly_res = poly_res.append(new_row,ignore_index=True)
    #return train_df,test_df
index += 1
    #f.lose()
#poly_rs.to_excel("../POLI/poly.xlsx",index= False)

------ TRAIN DATA ------

MSE : 293650.0005973578, RMSE: 541.894824294676, MAE : 87.03722736497548

R2 TRAIN 0.8325921731742727

TRAIN STD 1200.3486987386668

------ TEST DATA ------

MSE : 3229554.14300194, RMSE: 1797.096030545374, MAE : 342.6888225949348

R2 TEST -0.0809716558525011

TEST STD 941.3534568487511

--------------------------------

[0.01400955 0.02006761 0.01216134 0.02026677 0.03044795 0.00719963
 0.02421035 0.00604031 0.00801687 0.01998617 0.03254358 0.01089761
 0.21831706 0.01505818 0.00268136 0.02750279 0.02536306 0.02493319
 0.04433854 0.00244055 0.0039757  0.01180708 0.0240998  0.00236501
 0.00044349 0.30172123 0.01672885 0.00108655 0.00899434 0.01022315
 0.01796565 0.03410668]
------ TRAIN DATA ------

MSE : 293650.0005973578, RMSE: 541.894824294676, MAE : 87.03722736497548

R2 TRAIN 0.8325921731742727

TRAIN STD 1200.3486987386668

------ TEST DATA ------

MSE : 3229554.14300194, RMSE: 1797.096030545374, MAE : 342.6888225949348

R2 TEST -0.0809716558525011

TEST 

random forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
poly_res = pd.read_excel("./poly.xlsx")

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [total_no_corr_no_outliers_week.Total , israelis_no_corr_no_outliers_week.Israelis_Count , tourists_no_corr_no_outliers_week.Tourists_Count]
xs = [total_no_corr_no_outliers_week.drop(['Total' , 'Israelis_Count','Tourists_Count' ] , axis=1),
israelis_no_corr_no_outliers_week.drop(['Total' , 'Israelis_Count' ] , axis=1),
tourists_no_corr_no_outliers_week.drop(['Total' ,  'Tourists_Count' ] , axis=1)]

for x in xs:
    cols = x.columns
    drop = [x for x in cols if x in cols_to_drop]
    x.drop(drop,axis=1)

index = 0
for target in targets:
    
        print(target+"\n")
        #f = open("./"+target+"_general_UNI.txt" , "w")
        #to_drop = [x for x in targets if x != target]
        #print(to_drop)
        X = xs[index]
        
        y = ys[index]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)


        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()

        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)

        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        dtree = RandomForestRegressor(random_state = 987456 , n_estimators=5000,min_samples_leaf =50) 

        dtree.fit(X_train_scaled,y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = dtree.predict(X_train_scaled)
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #print(train_df.columns)
        y_test_pred_scaled = dtree.predict(X_test_scaled)
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        #print("Degrees: "+str(i)+",Target: "+target+"\n")
        print("------ TRAIN DATA ------\n")
        print("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        print("R2 TRAIN "+ str(r2_train)+"\n")
        print("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        print("------ TEST DATA ------\n")
        print("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        print("R2 TEST "+ str(r2_test)+"\n")
        print("TEST STD "+str(test_std)+"\n")
        print("--------------------------------\n")
        
        features = list(X.columns)
        fig = go.Figure()
        trace = go.Bar(
            x = dtree.feature_importances_,
            y = features,
            orientation='h'
        )
        # fig.add_trace(trace)
        # fig.write_image("./"+target+"_weekdays.png",width = 2000,height = 1500)
        
        # coeff = dtree.feature_importances_
        # print(coeff)

        #print("EQUATION : " + equation)
        # new_row = {'Descripton':'weekdays, no outliers + corr, no pollutions','Target':target,'Model_type':'Random Forests','Feature_Selection':'corr >= 0.4',
        # 'MAE_Training':train_mae,'MSE_Training':train_mse,
        # 'MSE_Training': train_rmse,'R2_Training': r2_train,
        # 'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
        # 'TEST_STD':test_std ,'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df),'PARAMS':dtree.get_params}

        # poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
        index += 1
    #f.lose()
# poly_res.to_excel("./poly.xlsx",index= False)

per site trees

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [df_no_outliers.Total , df_no_outliers.Israelis_Count , df_no_outliers.Tourists_Count]



for site in sites:
    index = 0
    print("-----"+site+"-----")
    for target in targets:
        print(target+"\n")
    #f = open("./"+target+"_general_UNI.txt" , "w")
    #to_drop = [x for x in targets if x != target]
    #print(to_drop)

    
        X = df_no_outliers.drop(['Total' , 'Israelis_Count', 'Tourists_Count' ] , axis=1)
        
        y = ys[index]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()

        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)

        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        dtree = DecisionTreeRegressor(random_state = 987456) 

        dtree.fit(X_train_scaled,y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = dtree.predict(X_train_scaled)
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #print(train_df.columns)
        y_test_pred_scaled = dtree.predict(X_test_scaled)
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        #print("Degrees: "+str(i)+",Target: "+target+"\n")
        print("------ TRAIN DATA ------\n")
        print("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        print("R2 TRAIN "+ str(r2_train)+"\n")
        print("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        print("------ TEST DATA ------\n")
        print("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        print("R2 TEST "+ str(r2_test)+"\n")
        print("TEST STD "+str(test_std)+"\n")
        print("--------------------------------\n")
        
        
        coeff = dtree.feature_importances_
        #print(coeff)

        #print("EQUATION : " + equation)
        #    new_row = {'Descripton':'general model ,Uni var feature selection, k=10, Degree: '+str(i),'Target':target,'Model_type':'POLY',
        #    'Degree' : str(i),'MAE_Training':train_mae,'MSE_Training':train_mse,
        # 'MSE_Training': train_rmse,'R2_Training': r2_train,
        # 'AE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
        # 'EST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}

        #poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    #f.lose()
#poly_rs.to_excel("../POLI/poly.xlsx",index= False)

In [ ]:
def drop(tmp, drops):
    cols = tmp.columns
    res = [x for x in cols if x in drops]
    return tmp.drop(res , axis = 1)
    

Site characteristics : trees

In [ ]:
water_sites = df.loc[df.is_water == 1]
arch_sites = df.loc[df.is_archaeology == 1]
water_sites = water_sites.dropna(axis = 0)
arch_sites = arch_sites.dropna(axis = 0)
#cleaning un relevant columns and stuff..
cols = ['Tourists_Count','Israelis_Count','Total','Date','Model_number','Site_Name']

water_sites_y = [water_sites.Total , water_sites.Israelis_Count,water_sites.Tourists_Count]
water_sites_x = drop(water_sites,cols)


arch_sites_y = [arch_sites.Total , arch_sites.Israelis_Count,arch_sites.Tourists_Count]
arch_sites_x = drop(arch_sites,cols)


In [101]:
north = no_pollution_df.loc[(no_pollution_df['region_North'] == 1) & (no_pollution_df['is_weekend'] == 0)]
central = no_pollution_df.loc[(no_pollution_df['region_Central'] == 1) & (no_pollution_df['is_weekend'] == 0)]
south = no_pollution_df.loc[(no_pollution_df['region_South'] == 1) & (no_pollution_df['is_weekend'] == 0)]

north.set_index('Date', drop=True , inplace=True)
central.set_index('Date', drop=True , inplace=True)
south.set_index('Date', drop=True , inplace=True)

cols = ['Tourists_Count','Israelis_Count','Total','Model_number','Site_Name']

north_y = [north.Total , north.Israelis_Count,north.Tourists_Count]
north_x = drop(north,cols)


central_y = [central.Total , central.Israelis_Count,central.Tourists_Count]
central_x = drop(central,cols)

south_y = [south.Total , south.Israelis_Count,south.Tourists_Count]
south_x = drop(south,cols)

In [102]:
targets = ['Total' , 'Israelis_Count']
regions = ['central','south' ,'north']

index = 0
for target in targets:
    
        print(target+" -  {} \n".format(regions[index]))

        X = north_x
        
        y = north_y[index]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)


        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()

        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)

        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        dtree = RandomForestRegressor(random_state = 987456 , n_estimators=1000,min_samples_leaf =50) 

        dtree.fit(X_train_scaled,y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = dtree.predict(X_train_scaled)
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #print(train_df.columns)
        y_test_pred_scaled = dtree.predict(X_test_scaled)
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)

        print("------ TRAIN DATA ------\n")
        print("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        print("R2 TRAIN "+ str(r2_train)+"\n")
        print("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        print("------ TEST DATA ------\n")
        print("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        print("R2 TEST "+ str(r2_test)+"\n")
        print("TEST STD "+str(test_std)+"\n")
        print("--------------------------------\n")


        index += 1

Total -  central 

------ TRAIN DATA ------

MSE : 530818.8496578876, RMSE: 728.573160127305, MAE : 399.8623653965607

R2 TRAIN -0.3069648225619803

TRAIN STD 362.682755280835

------ TEST DATA ------

MSE : 508481.5700541268, RMSE: 713.0789367623522, MAE : 390.57738559753847

R2 TEST -0.28737451644347334

TEST STD 366.58811862638566

--------------------------------

Israelis_Count -  south 

------ TRAIN DATA ------

MSE : 275618.3032685353, RMSE: 524.9936221217695, MAE : 274.0824656156651

R2 TRAIN -0.13096470085355127

TRAIN STD 205.23920171152153

------ TEST DATA ------

MSE : 255358.78672080956, RMSE: 505.3303738355825, MAE : 267.25039754313605

R2 TEST -0.12086208974878243

TEST STD 204.64271905139842

--------------------------------



In [93]:
import sys
sys.path.append("../Function")
import function

In [ ]:
function.plot_line(y_test_pred , y)